In [2]:
import numpy as np
import math as math

# Funckija koja racuna teziste skupa tacaka u 2D
def izracunaj_teziste(tacke):
  n = len(tacke)
  C_x = 0
  C_y = 0

  for tacka in tacke:
    C_x += tacka[0]
    C_y += tacka[1]

  return (C_x/n, C_y/n)

def izracunaj_prosek_rastojanja(tacke):
  rastojanje = 0
  n = len(tacke)
  for tacka in tacke:
    rastojanje += math.sqrt(tacka[0]**2 + tacka[1]**2)

  return rastojanje/n    
'''
 Funckija koja vrsi normalizaciju tako sto prvo svaku tacku u skupu translira za vektor CO
 gde je C teziste skupa tacaka, a O koordinatni pocetak. Zatim se vrsi skaliranje kooridnata
 tako da prosecno rastojanje skupa tacaka do koordinatnog pocetka bude sqrt(2). Ovako dobijen
 skup tacaka se naziva normalizovan.
'''
def normalizuj(tacke):

  (C_x, C_y) = izracunaj_teziste(tacke)
  
  nova_lista = []
  for tacka in tacke:
    x = tacka[0] - C_x
    y = tacka[1] - C_y
    nova_lista.append([x, y])

  lam = izracunaj_prosek_rastojanja(nova_lista)

  tacke = []

  for tacka in nova_lista:
    x = (tacka[0]/lam)*math.sqrt(2)
    y = (tacka[1]/lam)*math.sqrt(2)
    tacke.append([x, y])

  return tacke

tacke = [(7, 4), (6, 4), (5, 2), (0, 0), (-5, -5)]
tacke = normalizuj(tacke)
print(tacke)


[[1.2480836042156704, 0.8509660937834115], [0.9644282396211997, 0.8509660937834115], [0.6807728750267292, 0.2836553645944705], [-0.7375039479456232, -0.2836553645944705], [-2.1557807709179757, -1.701932187566823]]


In [29]:
def prebaci_u_projektivne(tacke):
  lista = []
  for tacka in tacke:
    x = tacka[0]
    y = tacka[1]
    lista.append([x, y, 1])
  return lista

# Funkcija koja formira matricu cije su kolone redom vektori A, B i C

def formiraj_matricu(A, B, C):
  array = np.array([A, B, C]).T
  return array

def izvuci_kolone(matrica):
  prva_kolona = []
  for i in range(3):
    prva_kolona.append(matrica[i][0])
  druga_kolona = []
  for i in range(3):
    druga_kolona.append(matrica[i][1])
  treca_kolona = []
  for i in range(3):
    treca_kolona.append(matrica[i][2])

  return prva_kolona, druga_kolona, treca_kolona

'''
  Ova funkcija predstavlja implementaciju naivnog algoritma za odredjivanje 
  matrice projektivnog preslikavanja za otklanjanje projektivne distorzije.
'''

def naivni(originali, slike):
  if (len(originali) != 4 or len(slike) != 4 or len(originali) != len(slike)):
    print("Funckija prima tacno 4 tacke!")
    return None

  originali = prebaci_u_projektivne(originali)
  slike = prebaci_u_projektivne(slike)
  
  A1, B1, C1, D1 = originali[0], originali[1], originali[2], originali[3]
  A2, B2, C2, D2 = slike[0], slike[1], slike[2], slike[3]
  D1 = np.array([D1]).T
  D2 = np.array([D2]).T

  matrica_1 = formiraj_matricu(A1, B1, C1)
  matrica_2 = formiraj_matricu(A2, B2, C2)
  
  # Potrbno je pronaci alfa, beta i gama iz izraza
  # D1 = alfa*A1 + beta*B1 + gama*C1
  # Slicno vazi i za kolone matrice matrica_2
  
  X = np.linalg.inv(matrica_1).dot(D1)
  Y = np.linalg.inv(matrica_2).dot(D2)

  prva_kolona, druga_kolona, treca_kolona = izvuci_kolone(matrica_1)
  P1 = formiraj_matricu(X[0]*prva_kolona, X[1]*druga_kolona, X[2]*treca_kolona)

  prva_kolona, druga_kolona, treca_kolona = izvuci_kolone(matrica_2)
  P2 = formiraj_matricu(Y[0]*prva_kolona, Y[1]*druga_kolona, Y[2]*treca_kolona)

  P = np.matmul(P2, np.linalg.inv(P1))
  return P

print(naivni([[-3, -1], [3, -1], [1, 1], [-1, 1]], [[-2, -1], [2, -1], [2, 1], [-2, 1]]))

[[ 2.  0.  0.]
 [ 0.  2. -1.]
 [ 0. -1.  2.]]


In [ ]:
'''
  Ova funkcija predstavlja implementaciju DLT algoritma za odredjivanje matrice
  projektivnog preslikavanja za otklanjanja projektivne distorzije
'''

def DLT(originali, slike):
  if(len(originali) < 4 or len(slike) < 4 or len(originali) != len(slike)):
    print("Losi argumenti funkcije!")
    return None

  n = len(originali)

  originali = prebaci_u_projektivne(originali)
  slike = prebaci_u_projektivne(slike)
  

